# 🔧 Solución de Errores Comunes - Fine-tuning

## Meta Day Uruguay 2025 - Módulo 4: Troubleshooting

Este notebook contiene soluciones para los errores más comunes al hacer fine-tuning con Unsloth y QLoRA.

### 🚨 Errores cubiertos:
1. **TypeError: Descriptors cannot be created directly** (protobuf)
2. **CUDA out of memory**
3. **ImportError: No module named 'unsloth'**
4. **RuntimeError: Expected all tensors to be on the same device**
5. **Problemas de versiones de dependencias**
6. **Errores de tokenización**

## 🔴 Error 1: TypeError - Descriptors cannot be created directly

**Error completo:**
```
TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
```

**Causa:** Incompatibilidad entre versiones de protobuf

**Solución:**

In [ ]:
# 🔧 SOLUCIÓN COMPLETA PARA ERROR DE PROTOBUF

# Método 1: Variable de entorno (más rápido pero menos eficiente)
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
print("✅ Variable de entorno configurada")

# Método 2: Downgrade de protobuf (recomendado)
print("🔄 Instalando versión compatible de protobuf...")
%pip install --upgrade "protobuf<=3.20.3" --quiet

# Verificar versión
import protobuf
print(f"📦 Protobuf version: {protobuf.__version__}")

# Reiniciar runtime si es necesario
print("⚠️ Si el error persiste, reinicia el runtime: Runtime > Restart Runtime")

## 🔴 Error 2: CUDA out of memory

**Error:** `RuntimeError: CUDA out of memory`

**Soluciones:**

In [ ]:
# 🔧 SOLUCIONES PARA MEMORIA GPU INSUFICIENTE

import torch
import gc

def clear_gpu_memory():
    """Limpiar memoria GPU"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print("🧹 Memoria GPU limpiada")
    else:
        print("ℹ️ No hay GPU disponible")

def check_gpu_memory():
    """Verificar uso de memoria GPU"""
    if torch.cuda.is_available():
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        allocated = torch.cuda.memory_allocated(0) / 1024**3
        cached = torch.cuda.memory_reserved(0) / 1024**3
        free = total - cached
        
        print(f"📊 Memoria GPU:")
        print(f"   Total: {total:.1f} GB")
        print(f"   Asignada: {allocated:.1f} GB")
        print(f"   En caché: {cached:.1f} GB")
        print(f"   Libre: {free:.1f} GB")
        
        if free < 2.0:
            print("⚠️ Poca memoria libre - considera reducir batch_size")
    else:
        print("ℹ️ No hay GPU disponible")

# Ejecutar diagnóstico
clear_gpu_memory()
check_gpu_memory()

print("\n💡 Soluciones si tienes poco memoria:")
print("1. Reducir max_seq_length (ej: 1024 en lugar de 2048)")
print("2. Reducir per_device_train_batch_size (ej: 2 en lugar de 8)")
print("3. Aumentar gradient_accumulation_steps para compensar")
print("4. Usar modelo más pequeño (7B en lugar de 8B)")
print("5. Activar gradient_checkpointing")

## 🔴 Error 3: ImportError - No module named 'unsloth'

**Solución de instalación paso a paso:**

In [ ]:
# 🔧 INSTALACIÓN PASO A PASO DE UNSLOTH

print("🔄 Instalando Unsloth y dependencias...")

# Paso 1: Limpiar instalaciones previas
%pip uninstall -y unsloth unsloth-zoo --quiet

# Paso 2: Instalar dependencias base
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet
%pip install transformers datasets accelerate peft --quiet

# Paso 3: Instalar bitsandbytes
%pip install bitsandbytes --quiet

# Paso 4: Instalar Unsloth
# Para Colab:
%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --quiet

# Para otros entornos, usar:
# %pip install unsloth --quiet

# Paso 5: Verificar instalación
try:
    from unsloth import FastLanguageModel
    print("✅ Unsloth instalado correctamente")
except ImportError as e:
    print(f"❌ Error al importar Unsloth: {e}")
    print("💡 Intenta reiniciar el runtime y ejecutar de nuevo")

## 🔴 Error 4: Expected all tensors to be on the same device

**Solución:**

In [ ]:
# 🔧 SOLUCIÓN PARA ERRORES DE DISPOSITIVO

import torch

def check_device_setup():
    """Verificar configuración de dispositivos"""
    print("🔍 Verificando configuración de dispositivos...")
    
    # Verificar CUDA
    cuda_available = torch.cuda.is_available()
    print(f"CUDA disponible: {cuda_available}")
    
    if cuda_available:
        device_count = torch.cuda.device_count()
        current_device = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(current_device)
        
        print(f"Dispositivos GPU: {device_count}")
        print(f"GPU actual: {current_device} ({device_name})")
        
        # Configurar dispositivo por defecto
        torch.cuda.set_device(0)
        print("✅ Dispositivo GPU configurado")
    else:
        print("⚠️ Usando CPU - el entrenamiento será más lento")

def fix_model_device(model, tokenizer):
    """Asegurar que modelo y tokenizer estén en el dispositivo correcto"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Mover modelo al dispositivo
    if hasattr(model, 'to'):
        model = model.to(device)
        print(f"✅ Modelo movido a {device}")
    
    # Configurar tokenizer
    if hasattr(tokenizer, 'pad_token') and tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print("✅ Pad token configurado")
    
    return model, tokenizer

# Ejecutar verificación
check_device_setup()

print("\n💡 Si el error persiste:")
print("1. Reinicia el runtime")
print("2. Asegúrate de usar FastLanguageModel.from_pretrained()")
print("3. No mezcles operaciones CPU/GPU manualmente")

## 🔴 Error 5: Conflictos de Versiones

**Instalación limpia con versiones compatibles:**

In [ ]:
# 🔧 INSTALACIÓN LIMPIA CON VERSIONES COMPATIBLES

print("🧹 Limpiando instalaciones previas...")

# Desinstalar paquetes problemáticos
packages_to_remove = [
    "unsloth", "unsloth-zoo", "xformers", "trl", 
    "transformers", "accelerate", "peft", "bitsandbytes"
]

for package in packages_to_remove:
    %pip uninstall -y {package} --quiet

print("📦 Instalando versiones compatibles...")

# Instalar versiones específicas compatibles
compatible_packages = {
    "protobuf": "<=3.20.3",
    "transformers": ">=4.36.0",
    "accelerate": ">=0.21.0",
    "peft": ">=0.4.0",
    "bitsandbytes": ">=0.41.0",
    "trl": "<0.9.0",
    "xformers": "<0.0.27"
}

for package, version in compatible_packages.items():
    print(f"Installing {package}{version}...")
    %pip install "{package}{version}" --quiet

# Instalar Unsloth al final
print("Installing Unsloth...")
%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --quiet

print("\n✅ Instalación limpia completada")
print("⚠️ IMPORTANTE: Reinicia el runtime antes de continuar")
print("   Runtime > Restart Runtime")

## 🔴 Error 6: Errores de Tokenización

**Problemas comunes con tokens y chat templates:**

In [ ]:
# 🔧 SOLUCIÓN PARA ERRORES DE TOKENIZACIÓN

def fix_tokenizer_issues(tokenizer):
    """Solucionar problemas comunes de tokenización"""
    print("🔧 Configurando tokenizer...")
    
    # Configurar pad token si no existe
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print("✅ Pad token configurado")
    
    # Configurar padding side
    tokenizer.padding_side = "right"
    print("✅ Padding side configurado")
    
    # Verificar tokens especiales
    special_tokens = {
        "bos_token": tokenizer.bos_token,
        "eos_token": tokenizer.eos_token,
        "pad_token": tokenizer.pad_token,
        "unk_token": tokenizer.unk_token
    }
    
    print("📋 Tokens especiales:")
    for name, token in special_tokens.items():
        print(f"   {name}: {token}")
    
    return tokenizer

def test_chat_template(tokenizer):
    """Probar chat template"""
    print("🧪 Probando chat template...")
    
    # Mensaje de prueba
    messages = [
        {"from": "human", "value": "Hola, ¿cómo estás?"},
        {"from": "gpt", "value": "¡Hola! Estoy bien, gracias por preguntar."}
    ]
    
    try:
        # Aplicar template
        formatted = tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=False
        )
        print("✅ Chat template funcionando")
        print("📝 Ejemplo formateado:")
        print(formatted[:200] + "...")
        
    except Exception as e:
        print(f"❌ Error en chat template: {e}")
        print("💡 Solución: Configurar template manualmente")

# Ejemplo de uso (descomenta cuando tengas un tokenizer)
# tokenizer = fix_tokenizer_issues(tokenizer)
# test_chat_template(tokenizer)

print("💡 Ejecuta estas funciones después de cargar tu tokenizer")

## ⚡ Soluciones Rápidas

### Comandos de emergencia:

In [ ]:
# 🚨 COMANDOS DE EMERGENCIA

def emergency_reset():
    """Reset completo del entorno"""
    print("🚨 RESET DE EMERGENCIA")
    
    # Limpiar memoria
    import gc
    import torch
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    # Variables de entorno
    import os
    os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    
    print("✅ Reset completado")
    print("⚠️ Reinicia el runtime si los problemas persisten")

def check_environment():
    """Verificar entorno completo"""
    print("🔍 DIAGNÓSTICO COMPLETO")
    print("=" * 40)
    
    # Python version
    import sys
    print(f"Python: {sys.version}")
    
    # PyTorch
    try:
        import torch
        print(f"PyTorch: {torch.__version__}")
        print(f"CUDA: {torch.cuda.is_available()} ({torch.version.cuda if torch.cuda.is_available() else 'N/A'})")
    except ImportError:
        print("❌ PyTorch no instalado")
    
    # Transformers
    try:
        import transformers
        print(f"Transformers: {transformers.__version__}")
    except ImportError:
        print("❌ Transformers no instalado")
    
    # Unsloth
    try:
        import unsloth
        print(f"Unsloth: ✅ Instalado")
    except ImportError:
        print("❌ Unsloth no instalado")
    
    # Protobuf
    try:
        import google.protobuf
        print(f"Protobuf: {google.protobuf.__version__}")
    except ImportError:
        print("❌ Protobuf no instalado")
    
    print("=" * 40)

# Ejecutar diagnóstico
check_environment()
emergency_reset()

## 🛡️ Prevención de Errores

### Mejores prácticas para evitar problemas:

In [ ]:
# 🛡️ MEJORES PRÁCTICAS

def setup_best_practices():
    """Configuración preventiva"""
    print("🛡️ Configurando mejores prácticas...")
    
    import os
    import warnings
    
    # Variables de entorno preventivas
    env_vars = {
        "PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION": "python",
        "TOKENIZERS_PARALLELISM": "false",
        "CUDA_LAUNCH_BLOCKING": "1",  # Para debugging CUDA
        "PYTHONWARNINGS": "ignore",   # Reducir warnings
    }
    
    for key, value in env_vars.items():
        os.environ[key] = value
        print(f"✅ {key} = {value}")
    
    # Suprimir warnings comunes
    warnings.filterwarnings("ignore", category=UserWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    print("\n📋 CHECKLIST ANTES DE ENTRENAR:")
    checklist = [
        "✅ Protobuf <= 3.20.3 instalado",
        "✅ Variables de entorno configuradas",
        "✅ GPU memory < 80% de uso",
        "✅ Batch size apropiado para tu GPU",
        "✅ Dataset cargado correctamente",
        "✅ Chat template configurado",
        "✅ Modelo y tokenizer en mismo dispositivo"
    ]
    
    for item in checklist:
        print(f"   {item}")
    
    print("\n💡 TIPS ADICIONALES:")
    tips = [
        "Siempre reinicia runtime después de instalar paquetes",
        "Usa versiones específicas en lugar de 'latest'",
        "Guarda checkpoints frecuentemente",
        "Monitorea uso de memoria durante entrenamiento",
        "Prueba con datasets pequeños primero"
    ]
    
    for tip in tips:
        print(f"   💡 {tip}")

# Ejecutar configuración
setup_best_practices()

## 🎯 Resumen de Soluciones

### 🔧 Orden recomendado para solucionar problemas:

1. **Ejecutar emergency_reset()** - Limpia memoria y configura variables
2. **Instalar protobuf <= 3.20.3** - Soluciona el error más común
3. **Reinstalar Unsloth** - Con versiones compatibles
4. **Reiniciar runtime** - Aplicar cambios
5. **Verificar GPU memory** - Ajustar batch_size si es necesario
6. **Probar con dataset pequeño** - Validar configuración

### 📞 Si nada funciona:
- Usa Google Colab Pro para más memoria
- Prueba con CPU (más lento pero funcional)
- Considera usar modelos más pequeños
- Revisa la documentación oficial de Unsloth

### 🆘 Recursos de ayuda:
- [Unsloth GitHub Issues](https://github.com/unslothai/unsloth/issues)
- [Hugging Face Forum](https://discuss.huggingface.co/)
- [PyTorch Troubleshooting](https://pytorch.org/docs/stable/notes/faq.html)

---
**Meta Day Uruguay 2025** - Módulo 4: Troubleshooting 🇺🇾